In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
data_2020 = pd.read_csv('/Users/chocz/Downloads/data1.csv')
data_2019 = pd.read_csv('/Users/chocz/Downloads/medium_2019.csv')

/Users/chocz/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (6) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
data = data_2019.append(data_2020)

In [8]:
df = data[(data.tag == 'Religion') | (data.tag == 'Art') | (data.tag == 'Venture-Capital') | (data.tag =='Language') | (data.tag == 'Science')] 

In [9]:
df.index = np.arange(1, len(df)+1)
df['index'] = np.arange(1, len(df)+1)

<ipython-input-9-90d60b54eed6>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['index'] = np.arange(1, len(df)+1)


In [10]:
df.astype({'title':'string','subtitle':'string','tag':'string'}).dtypes

Unnamed: 0       int64
article_id       int64
article_url     object
title           string
subtitle        string
claps            int64
responses       object
reading_time     int64
tag             string
date            object
index            int64
dtype: object

In [11]:
df['subtitle'].isnull().values.any()

True

In [12]:
genre_popularity = (df.tag.str.split('|')
                      .explode()
                      .value_counts()
                      .sort_values(ascending=False))
genre_popularity.head(10)

Art                3940
Science            3164
Religion           1530
Venture-Capital    1521
Language            781
Name: tag, dtype: int64

In [20]:
import wordcloud
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from itertools import combinations
import seaborn as sns
from sklearn.feature_extraction.text import TfidfVectorizer

In [61]:
s = "Sister Corita Against the System"
tf_wrong = TfidfVectorizer(analyzer='word', ngram_range=(1,2))
tf_wrong.fit([s])
tf_wrong.get_feature_names()
df['combined features'] = str(df['tag'])+str(df['title'])

<ipython-input-61-9a0a37d461e3>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['combined features'] = str(df['tag'])+str(df['title'])


In [62]:
[c for i in range(1,2) for c in combinations(s.split(), r=i)]

[('Sister',), ('Corita',), ('Against',), ('the',), ('System',)]

In [63]:
tf = TfidfVectorizer(analyzer=lambda s: (c for i in range(1,4)
                                             for c in combinations(s.split('|'), r=i)))
tfidf_matrix = tf.fit_transform(df['tag'])
tfidf_matrix.shape
# (3883, 353)
tfidf_matrix

<10936x5 sparse matrix of type '<class 'numpy.float64'>'
	with 10936 stored elements in Compressed Sparse Row format>

In [64]:
pd.DataFrame(tfidf_matrix.todense(), columns=tf.get_feature_names(), index=df.title).sample(5, axis=1).sample(10, axis=0)

,"(Art,)","(Language,)","(Science,)","(Venture-Capital,)","(Religion,)"
title,,,,,
Questions from the Cosmos,0.0,0.0,1.0,0.0,0.0
Twist and Fade,1.0,0.0,0.0,0.0,0.0
"Fra Angelico, the Angelic Friar",1.0,0.0,0.0,0.0,0.0
COVID and AIDS,0.0,0.0,0.0,0.0,1.0
Winter Holidays Hurt LGBTQ People: You Can Help,0.0,0.0,0.0,0.0,1.0
The Painting,1.0,0.0,0.0,0.0,0.0
Sarah Belclaire — Ophelia Risen from the Lake,1.0,0.0,0.0,0.0,0.0
Indian Paintings | Buy Indian Art: A Brief Introduction — IndianArtIdeas,1.0,0.0,0.0,0.0,0.0
"The Weekly Roundup: Translation devices changing the business game, machine learning aiding scientific research and…",0.0,0.0,1.0,0.0,0.0


In [65]:
from sklearn.metrics.pairwise import cosine_similarity
cosine_sim = cosine_similarity(tfidf_matrix)

In [66]:
cosine_sim_df = pd.DataFrame(cosine_sim, index=df['title'], columns=df['title'])
print('Shape:', cosine_sim_df.shape)
cosine_sim_df.sample(5, axis=1).round(2)

Shape: (10936, 10936)


title,How Does This Candle Suck Water Up Into a Glass?,"Meet The Blob, A Creature That Has Almost 720 Sexes — But No Brain",Comment lever des fonds pour une startup ? [de A à Z],One of the Universe’s rarest events captured by Dark Matter detectors,KULTURA — The Unmixtakable
title,,,,,
Capturing Little Russia Before It ‘Disappears’,0.0,0.0,0.0,0.0,1.0
Sister Corita Against the System,0.0,0.0,0.0,0.0,1.0
Let’s kick off the world’s largest study into arts and health!,0.0,0.0,0.0,0.0,1.0
"<strong class=""markup--strong markup--h3-strong"">Treasure Island’s New Sculpture, Cut from Old Bay Bridge Steel, Signals Massive Changes Ahead</strong>",0.0,0.0,0.0,0.0,1.0
"<strong class=""markup--strong markup--h3-strong"">THE GHOST AND MS. TAGGART</strong>",0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...
How Art Is Saving Our Lives During Covid-19,1.0,1.0,0.0,1.0,0.0
"To, The Significant Other",1.0,1.0,0.0,1.0,0.0
Nomenclature of hydrocarbons,1.0,1.0,0.0,1.0,0.0


In [67]:
def genre_recommendations(i, M, items, k=5):
    """
    Recommends movies based on a similarity dataframe

    Parameters
    ----------
    i : str
        Movie (index of the similarity dataframe)
    M : pd.DataFrame
        Similarity dataframe, symmetric, with movies as indices and columns
    items : pd.DataFrame
        Contains both the title and some other features used to define similarity
    k : int
        Amount of recommendations to return

    """
    ix = M.loc[:,i].to_numpy().argpartition(range(-1,-k,-1))
    closest = M.columns[ix[-1:-(k+2):-1]]
    closest = closest.drop(i, errors='ignore')
    return pd.DataFrame(closest).merge(items).head(k)

In [71]:
df[df.title.eq('To, The Significant Other')]

,Unnamed: 0,article_id,article_url,title,subtitle,claps,responses,reading_time,tag,date,index,combined features
10933,62705,62706,https://medium.com/@asiffarhankhan/to-the-sign...,"To, The Significant Other",NaN,0,0,3,Science,2020-04-14,10933,1 Art\n2 Art\n3 ...


In [72]:
genre_recommendations('Nomenclature of hydrocarbons', cosine_sim_df, df[['title', 'article_url','tag']])

,title,article_url,tag
0,Diet or Exercise for Maintaining Weight Loss? ...,https://medium.com/@gr8er-self-care/diet-or-ex...,Science
1,"That device exists, and it’s called the Fitbit...",https://medium.com/@pkha_40779/that-device-exi...,Science
2,"Essence, the study of the essence we all need ...",https://medium.com/environmental-psychology/es...,Science
3,Deciphering the new CBD Products,https://medium.com/@susanhozak/deciphering-the...,Science
4,The console object is very much useful for the,https://medium.com/@pkha_40779/the-console-obj...,Science


In [73]:
df[df.title.eq('Capturing Little Russia Before It ‘Disappears’')]

,Unnamed: 0,article_id,article_url,title,subtitle,claps,responses,reading_time,tag,date,index,combined features
1,420,421,https://thebolditalic.com/exploring-little-rus...,Capturing Little Russia Before It ‘Disappears’,A local artist explores a vibrant San Francisc...,554,5,6,Art,2019-10-30,1,1 Art\n2 Art\n3 ...


In [74]:
genre_recommendations('Capturing Little Russia Before It ‘Disappears’', cosine_sim_df, df[['title', 'article_url','tag']])

,title,article_url,tag
0,"<strong class=""markup--strong markup--h3-stron...",https://medium.com/@letmeconfess/why-do-i-danc...,Art
1,The Timeline of Color in Art,https://medium.com/the-collector/the-timeline-...,Art
2,The Portion Gallery: Reimagined,https://medium.com/portion/the-portion-gallery...,Art
3,Dysmorphia,https://medium.com/@abrahamwoodliff/dysmorphia...,Art
4,Respect Your “Kin”,https://medium.com/minneapolis-institute-of-ar...,Art
